# Security

* This notebook uses Application Default Credentials (ADC) for BigQuery authentication.
* Authenticate with gcloud before running: `gcloud auth application-default login`
* No API key files needed - authentication is handled through your gcloud session.

# Tools

In [ ]:
# Package and library installation
packages_needed = c("tidyverse", "knitr") # comma delimited vector of package names
packages_installed = packages_needed %in% rownames(installed.packages())

if (any(! packages_installed))
  install.packages(packages_needed[! packages_installed])
for (i in 1:length(packages_needed)) {
  library(packages_needed[i], character.only = T)
}

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



* Using Application Default Credentials - no manual key file needed if you've run `gcloud auth application-default login`

In [ ]:
install.packages("bigrquery")
library(bigrquery)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Rcpp’, ‘rapidjsonr’




# Source

## Database Connection

In [ ]:
# BigQuery Authentication using Application Default Credentials
# This will use your gcloud authentication automatically
# Run this first in terminal: gcloud auth application-default login
bq_auth()

In [ ]:
Sys.setenv(BIGQUERY_TEST_PROJECT = "mpg-data-warehouse")

In [ ]:
billing <- bq_test_project()

### vegetation_point_intercept_gridVeg

In [ ]:
con_point_intercept <- dbConnect(
  bigrquery::bigquery(),
  project = "mpg-data-warehouse",
  dataset = "vegetation_point_intercept_gridVeg",
  billing = billing
)

In [ ]:
dbListTables(con_point_intercept)

[1] "gridVeg_additional_species"         "gridVeg_ground_cover_metadata"     
[3] "gridVeg_image_metadata"             "gridVeg_point_intercept_ground"    
[5] "gridVeg_point_intercept_vegetation" "gridVeg_shrub_tree"                
[7] "gridVeg_survey_metadata"

In [ ]:
ground_point_intercept_sql <-
"
  SELECT
    survey_ID,
    grid_point,
    intercept_ground_code,
    COUNT(intercept_ground_code) / 2 AS intercepts_pct
  FROM
    `mpg-data-warehouse.vegetation_point_intercept_gridVeg.gridVeg_point_intercept_ground`
  GROUP BY
    survey_ID, grid_point, intercept_ground_code
"
bq_ground_point_intercept <- bq_project_query(billing, ground_point_intercept_sql)
tb_ground_point_intercept <- bq_table_download(bq_ground_point_intercept)
# Drop NA rows. These only occur in intercept_ground_code, are always associated with
# zero values in intercepts_pct, and will be expanded to true values later using complete()
# df_ground_point_intercept <- as.data.frame(tb_ground_point_intercept) %>% drop_na() %>% glimpse() ## LINE COMMENTED OUT; LINE BELOW ADDED; BL; 2021-01-22
df_ground_point_intercept <- as.data.frame(tb_ground_point_intercept) %>% filter(intercept_ground_code != "NA") %>% glimpse()

Rows: 9,599
Columns: 4
$ survey_ID             <chr> "5FC77E15-5632-46C2-9AC1-8F9ECA9C5515", "5FC77E1…
$ grid_point            <int> 512, 512, 512, 512, 257, 257, 257, 257, 257, 513…
$ intercept_ground_code <chr> "L", "BG", "M", "BV", "L", "WDT", "OTHER", "BV",…
$ intercepts_pct        <dbl> 63.5, 11.5, 18.0, 1.5, 73.0, 0.5, 8.0, 3.0, 14.5…


### gridVeg_survey_metadata

In [ ]:
survey_metadata_sql <-
"
  SELECT
    survey_ID,
    year,
    survey_sequence
  FROM
    `mpg-data-warehouse.vegetation_point_intercept_gridVeg.gridVeg_survey_metadata`
"
bq_survey_metadata <- bq_project_query(billing, survey_metadata_sql)
tb_survey_metadata <- bq_table_download(bq_survey_metadata)
df_survey_metadata <- as.data.frame(tb_survey_metadata) %>% glimpse()

Rows: 1,615
Columns: 3
$ survey_ID       <chr> "138", "139", "135", "134", "137", "136", "141", "143"…
$ year            <int> 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, …
$ survey_sequence <chr> "2010", "2010", "2010", "2010", "2010", "2010", "2010"…


### ground_cover_metadata

In [ ]:
sql_ground_meta <-
"
SELECT
  intercept_ground_code,
  ground_group
FROM
  `mpg-data-warehouse.vegetation_point_intercept_gridVeg.gridVeg_ground_cover_metadata`
"
bq_ground_meta <- bq_project_query(billing, sql_ground_meta)
tb_ground_meta <- bq_table_download(bq_ground_meta)
df_ground_meta <- as.data.frame(tb_ground_meta) %>% arrange(intercept_ground_code)

df_ground_meta %>% kable(format = "pandoc")



intercept_ground_code   ground_group 
----------------------  -------------
BG                      inorganic    
BV                      vas_plant    
G                       inorganic    
L                       litter       
LIC                     lichen       
M                       nonvas_plant 
M/L                     nonvas_plant 
OTHER                   inorganic    
R                       inorganic    
SC                      dung         
SD                      dung         
SE                      dung         
SH                      dung         
SU                      dung         
WDL                     woody        
WDS                     woody        
WDSTUMP                 woody        
WDT                     woody        

# Wrangle

In [ ]:
# show unique intercept ground codes
df_ground_point_intercept %>%
  distinct(intercept_ground_code) %>%
  arrange(intercept_ground_code)

intercept_ground_code
<chr>
BG
BV
G
L
LIC
M
M/L
OTHER
R


## Complete intercept_ground_code
- Complete the data frame for `intercept_ground_code` and fill null cells with 0
- As of 2020-08-24: with 18 ground cover codes and 1244 survey IDs, we should end up with 22,392 rows after completeing the data frame
- We should also end up with 1244 distinct combinations of survey_ID and grid_point

In [ ]:
df_ground_point_intercept_fill <-
  df_ground_point_intercept %>%
  complete(intercept_ground_code, nesting(survey_ID, grid_point), fill= list(intercepts_pct = 0)) %>%
  glimpse()

df_ground_point_intercept_fill %>%
  distinct(survey_ID, grid_point) %>%
  count()

Rows: 24,966
Columns: 4
$ intercept_ground_code <chr> "BG", "BG", "BG", "BG", "BG", "BG", "BG", "BG", …
$ survey_ID             <chr> "012C5FAD-2451-41B0-9E2F-432D1ECEB55C", "0133805…
$ grid_point            <int> 285, 505, 401, 23, 305, 323, 142, 511, 402, 196,…
$ intercepts_pct        <dbl> 15.5, 14.5, 16.5, 16.5, 1.5, 0.0, 7.0, 0.0, 19.5…


n
<int>
1387


## Join datasets

In [ ]:
df_gridVeg_groundCover_intercepts_join <-
  df_ground_point_intercept_fill %>%
  left_join(df_survey_metadata, by = "survey_ID") %>%
  left_join(df_ground_meta, by = "intercept_ground_code") %>%
  select(survey_ID, year, survey_sequence, grid_point, intercept_ground_code, ground_group, intercepts_pct) %>%
  glimpse()

Rows: 24,966
Columns: 7
$ survey_ID             <chr> "012C5FAD-2451-41B0-9E2F-432D1ECEB55C", "0133805…
$ year                  <int> 2016, 2016, 2017, 2016, 2021, 2015, 2021, 2016, …
$ survey_sequence       <chr> "2016", "2016", "2017", "2016", "2021", "2015", …
$ grid_point            <int> 285, 505, 401, 23, 305, 323, 142, 511, 402, 196,…
$ intercept_ground_code <chr> "BG", "BG", "BG", "BG", "BG", "BG", "BG", "BG", …
$ ground_group          <chr> "inorganic", "inorganic", "inorganic", "inorgani…
$ intercepts_pct        <dbl> 15.5, 14.5, 16.5, 16.5, 1.5, 0.0, 7.0, 0.0, 19.5…


## Filter

In [ ]:
# df_gridVeg_groundCover_intercepts
# df_gridVeg_groundCover_intercepts_join
df_gridVeg_groundCover_intercepts <- df_gridVeg_groundCover_intercepts_join %>%
  filter(year == 2022 & grid_point != 586) %>%
  glimpse()

Rows: 882
Columns: 7
$ survey_ID             <chr> "08CD58F4-828D-47B9-965C-C6E04CE02350", "0C37F6E…
$ year                  <int> 2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022, …
$ survey_sequence       <chr> "2022", "2022", "2022", "2022", "2022", "2022", …
$ grid_point            <int> 203, 357, 180, 513, 309, 572, 317, 342, 536, 257…
$ intercept_ground_code <chr> "BG", "BG", "BG", "BG", "BG", "BG", "BG", "BG", …
$ ground_group          <chr> "inorganic", "inorganic", "inorganic", "inorgani…
$ intercepts_pct        <dbl> 15.0, 3.5, 0.5, 3.5, 9.0, 0.0, 5.5, 2.5, 24.5, 1…


## QC
* No rows contain missing data, and this makes sense since `complete()` was used to fill response variables.
* Numeric variables appear correct in the summary

In [ ]:
df_gridVeg_groundCover_intercepts[which(!complete.cases(df_gridVeg_groundCover_intercepts)), ]

survey_ID,year,survey_sequence,grid_point,intercept_ground_code,ground_group,intercepts_pct
<chr>,<int>,<chr>,<int>,<chr>,<chr>,<dbl>


In [ ]:
summary(df_gridVeg_groundCover_intercepts)

  survey_ID              year      survey_sequence      grid_point   
 Length:882         Min.   :2022   Length:882         Min.   : 45.0  
 Class :character   1st Qu.:2022   Class :character   1st Qu.:109.0  
 Mode  :character   Median :2022   Mode  :character   Median :306.0  
                    Mean   :2022                      Mean   :293.4  
                    3rd Qu.:2022                      3rd Qu.:466.0  
                    Max.   :2022                      Max.   :582.0  
 intercept_ground_code ground_group       intercepts_pct  
 Length:882            Length:882         Min.   : 0.000  
 Class :character      Class :character   1st Qu.: 0.000  
 Mode  :character      Mode  :character   Median : 0.000  
                                          Mean   : 5.556  
                                          3rd Qu.: 1.500  
                                          Max.   :98.000  

In [ ]:
df_gridVeg_groundCover_intercepts %>%
  group_by(year) %>%
  summarize(count_distinct = n_distinct(survey_ID))

year,count_distinct
<int>,<int>
2022,49


# Output

In [ ]:
# Output 2022-08-17 esamsoe
# write_csv(df_gridVeg_groundCover_intercepts, file = "gridVeg_groundCover_intercepts_WRANGLE.csv")